In [13]:
# imports
import requests
import pandas as pd

In [14]:
# function to make flexible the url to explore de Api directories
def url_fun(*args):
    url = 'http://gateway.marvel.com/'
    return url + '/'.join(args)

# get credentials
def get_key(key):
    with open('C:/Windows/AppsKeys/Marvel/'+key+'.key', 'r') as file_read:
        app_key = file_read.readline()
    return app_key

# get data from url
def get_data_from_url():
    # get credentials
    apikey = get_key('apikey')
    ts = get_key('ts')
    hash = get_key('hash')
    # get url full path
    url = url_fun('v1', 'public', 'characters')
    # request data from url
    args = {'apikey': apikey,
            'ts': ts,
            'hash': hash}
    headers = {'Content-Type': 'application/json'}
    response = requests.get(url, params=args, headers=headers)
    # built a dictionary object with response
    print(response.status_code)
    if response.status_code == 200:
        response = response.json()
    # return dictionary object
    return response


In [15]:
# save response into dictionay
response_dict = get_data_from_url()
# save data results into a list 
list_data_results = response_dict['data']['results']


200


In [16]:
# get useful data from response in order to built de tabular data
heros = {"data":[]}

def add_hero(hero):
    heros["data"].append(hero)

for hero in list_data_results:
    imageUrl = ''
    imageUrl += hero['thumbnail']['path']
    imageUrl += '.'
    imageUrl += hero['thumbnail']['extension']
    item = {}
    item['hero_id'] = hero['id']
    item['hero_name'] = hero['name']
    item['hero_desc'] = hero['description']
    item['modified'] = hero['modified']
    item['hero_img'] = imageUrl
    item['hero_url'] = hero['urls'][0]['url']
    add_hero(item)

In [17]:
# built pandas tabular data with de dictironary useful data
df_headers = ('Id','Name','Description','Modified','Image URL','URL')
df_heros = pd.DataFrame(heros['data'][0].values()).transpose()
for i in range(1, len(list_data_results)):
    df_next = pd.DataFrame(heros['data'][i].values()).transpose()
    df_heros = df_heros.append(df_next, ignore_index=True)
df_heros.columns = df_headers

In [18]:
# save tabular data into a csv file
df_heros.to_csv("../outputs/marvel-heros.csv")